# For a set of csv files, read them in one at a time, perform t-SNE followed by DBSCAN, save figures and results 

In [2]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import decomposition, metrics
from sklearn.preprocessing import scale, robust_scale
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE

In [5]:
#directory = '/Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/merged_csvs/'
directory = '/Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/screen_357_cell_plate_1_labeled_merged_csvs/'
df = []
do_boxplots = 0; # binary, 1 indicates that boxplots should be made
#for filename in glob.glob(directory + "LABELLED*.csv"): # reads through all files in this directory looking for *.csv and ignores sub-directories
for filename in glob.glob(directory + "LABELLED*n22*.csv"): # reads through all files in this directory looking for *.csv and ignores sub-directories
    print("Read in" + filename)
    # Set some variables equal to "None" so that I don't have surprises later
    my_data=None; my_data_headers=None; meta_headers=None; my_data_data_headers=None
    my_scaled_data = None; word_as_num = None; tsne_out_mink = None; labels = None
    
    my_data=pd.read_csv(filename, index_col=0)
    my_data_headers = list(my_data)
    meta_headers = ["Width","cell_label","cell_plate","lineage","screen","well","Time"]
    my_data_data_headers = [x for x in my_data_headers if not x in meta_headers]
    # Scale the data columns
    my_scaled_data = scale(my_data[my_data_data_headers])
    # Make a list where cell_label is converted to numbers for plotting
    word_as_num=[]
    for word in my_data['cell_label']:
        if word == "unlabelled":
            word_as_num.append("0")
        if word == "blast":
            word_as_num.append("0.5")
    lr = 3000 # Set the learning rate for t-SNE
    # Perform t-SNE
    print("Starting t-SNE calculation using this many data points: " + str(my_scaled_data.shape[0]))
    tsne_out_mink = TSNE(metric='minkowski', learning_rate=lr, n_iter=lr, random_state=11).fit_transform(my_scaled_data)
    print("Done with t-SNE calculation")
    # Plot and save the output
    plt.figure(figsize=(20, 10))
    plt.gcf().clear()
    plt.subplot(121)
    plt.scatter(tsne_out_mink[:, 0], tsne_out_mink[:, 1], c= word_as_num, cmap=plt.cm.viridis)
    plt.savefig(filename + "__tsne.png")
    plt.close()
    # Do DBSCAN on the tSNEd data
    print("Starting DBSCAN")
    # Set DBSCAN eps hyper-parameter automatically
    eps_ = float(0.6)*float(11000)/float(my_scaled_data.shape[0])
    dbsc = DBSCAN(eps = eps_, min_samples=10).fit(tsne_out_mink)
    labels = dbsc.labels_
    core_samples = np.zeros_like(labels, dtype = bool)
    core_samples[dbsc.core_sample_indices_] = True
    unique_labels = np.unique(labels)
    print("Plotting DBSCAN with eps = to " + str(eps_))
    # Plot the DBSCAN results
    plt.figure(figsize=(20, 10))
    plt.gcf().clear()
    plt.subplot(121)
    col = np.linspace(0,1,len(unique_labels))
    for i in xrange(len(unique_labels)):
        plt.plot(tsne_out_mink[np.where(labels==unique_labels[i])[0], 0], 
             tsne_out_mink[np.where(labels==unique_labels[i])[0], 1],
                #'o', mfc=plt.cm.viridis(col[i]), label=unique_labels[i])
                'o', mfc=plt.cm.nipy_spectral(col[i]), label=unique_labels[i])
                #'o', mfc=plt.cm.prism(col[i]), label=unique_labels[i])
                #'o', mfc=plt.cm.flag(col[i]), label=unique_labels[i])
    plt.legend(loc='best', frameon=False)
    plt.savefig(filename + "__DBSCAN_eps" + str(eps_) +"_of_tsne.png")
    plt.close()
    # Summarize the combination of tSNE and DBSCAN results
    tsne_db = [] # This will be a list of lists where the 1st entry is the cluster ID, 2nd: is
    ## how many "unlabelled" are in that cluster, 3rd: number of blast, 4th: number of healthy.
    for i in xrange(len(unique_labels)):
        clst_index = my_data['cell_label'][np.where(labels==unique_labels[i])[0]]
        if clst_index.empty == True:
            print("Cluster index and original data don't line up right. STOP AND FIX")
            break
        tsne_db.append([unique_labels[i],sum(clst_index=='unlabelled'),
                         sum(clst_index=='blast')])
    print(tsne_db)
    # Prepare to plot the results as grouped barplots
    print("Plotting grouped barplots")
    tsne_db_df = pd.DataFrame(tsne_db,columns=["Cluster Label","Unlabelled","Blast"])
    tsne_db_df_melted = pd.melt(tsne_db_df,value_vars=["Unlabelled","Blast"],id_vars="Cluster Label")
    plt.gcf().clear()
    ax = sns.barplot(hue="variable",y="value",x="Cluster Label",data=tsne_db_df_melted,log='y')
    plt.savefig(filename + '__barplot.png')
    plt.close()
    if do_boxplots == 1:
        # For each cluster plot boxplots of each feature
        print("Starting boxplots plotting")
        for i in xrange(len(unique_labels)):
            clst_index = my_data.ix[np.where(labels==unique_labels[i])[0],my_data_data_headers]
            clst_index_scaled = my_scaled_data[np.where(labels==unique_labels[i])[0],]
            # Plot the unscaled data
            locations = range(1,(len(my_data_data_headers)+1))
            plt.figure()
            plt.gcf().clear()
            plt.boxplot(clst_index.as_matrix(),positions=locations)
            plt.title("Cluster number " + str(unique_labels[i]))
            plt.ylabel('A.U.')
            plt.xticks(locations, my_data_data_headers,rotation='vertical')
            #plt.yscale('log')
            percs = (clst_index.describe(percentiles=[0.1,0.5,0.9]))
            plt.ylim(percs.iloc[4].min(),percs.iloc[6].max())
            #plt.show()
            plt.savefig(filename + "__boxplots_of_cluster" +str(unique_labels[i])+".png")
            plt.close()
            locations = range(1,(len(my_data_data_headers)+1))
            # Plot the scaled data
            plt.figure()
            plt.gcf().clear()
            plt.boxplot(clst_index_scaled,positions=locations)
            plt.title("Cluster number " + str(unique_labels[i]))
            plt.ylabel('A.U.')
            plt.xticks(locations, my_data_data_headers,rotation='vertical')
            #plt.yscale('log')
            percs_scaled = pd.DataFrame(clst_index_scaled).describe(percentiles=[0.1,0.5,0.9])
            plt.ylim(percs_scaled.iloc[4].min(),percs_scaled.iloc[6].max())
            plt.savefig(filename + "__scaled_boxplots_of_cluster" + str(unique_labels[i]) + ".png")
            plt.close()
    print("Done with " + filename)

Read in/Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/screen_357_cell_plate_1_labeled_merged_csvs/LABELLED_by_cell_type_screen_357_cell_plate_1_labeled_merged_screen_357_cell_plate_1_labeled_merged_well_n22.csv
Starting t-SNE calculation using this many data points: 12680
Done with t-SNE calculation
Starting DBSCAN
Plotting DBSCAN with eps = to 0.520504731861
[[-1, 46, 2], [0, 98, 3], [1, 11086, 7], [2, 10, 0], [3, 347, 0], [4, 20, 0], [5, 15, 0], [6, 11, 0], [7, 85, 0], [8, 209, 0], [9, 43, 0], [10, 335, 0], [11, 206, 0], [12, 20, 0], [13, 117, 0], [14, 10, 0], [15, 10, 0]]
Plotting grouped barplots
Done with /Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/screen_357_cell_plate_1_labeled_merged_csvs/LABELLED_by_cell_type_screen_357_cell_plate_1_labeled_merged_screen_357_cell_plate_1_labeled_merged_well_n22.csv


# Add the following code to the above 

In [ ]:
#### I started to work on this, but didn't finish. Psuedo code is mostly done.
'''
#### Export the labelled populations of a well
# Find the fcs file that matches the csv file. 
## First get rid of the .csv, then get rid of the path and the LABELLED,
## now add the path back to the core name identified above, check that 
## it only matches one file, print matches for user to verify
output_name = filename.split(".csv")[0]
filename_fcs = 
for i in xrange(len(unique_labels)):
    fcs_file = FCSParser(path)
    #cluster_data = my_data_keepers.iloc[np.where(labels==unique_labels[i])[0]].astype('float')
    #new_fcs_file = fcs_file.clone(data=cluster_data.as_matrix().astype('float'))
    new_fcs_file = fcs_file.clone(data= fcs_file.data[np.where(labels==unique_labels[i])[0]].astype('float'))
    new_fcs_file.annotation['cluster_label'] = unique_labels[i]
    #new_fcs_file.write_to_file("/Users/tswenson/Documents/Joels/TEST.fcs")
    new_fcs_file.write_to_file(output_name + '__cluster_' + str(unique_labels[i])\
                               + '_exported_'\
                               + datetime.datetime.now().strftime('%Y.%m.%d %I.%M.%S')\
                               + '.fcs')
'''

In [ ]:
'''
# Export Well C3 or my_data
path = '/Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/training_set/screen_357_cell_plate_1_well_C3_original.fcs'
output_name = path.split(".fcs")[0]
for i in xrange(len(unique_labels)):
    fcs_file = FCSParser(path)
    #cluster_data = my_data_keepers.iloc[np.where(labels==unique_labels[i])[0]].astype('float')
    #new_fcs_file = fcs_file.clone(data=cluster_data.as_matrix().astype('float'))
    new_fcs_file = fcs_file.clone(data= fcs_file.data[np.where(labels==unique_labels[i])[0]].astype('float'))
    new_fcs_file.annotation['cluster_label'] = unique_labels[i]
    #new_fcs_file.write_to_file("/Users/tswenson/Documents/Joels/TEST.fcs")
    new_fcs_file.write_to_file(output_name + '__cluster_' + str(unique_labels[i])\
                               + '_exported_'\
                               + datetime.datetime.now().strftime('%Y.%m.%d %I.%M.%S')\
                               + '.fcs')
'''

In [7]:
### Only plot clusters with more than 50 events
tsne_db_df_50 = tsne_db_df[tsne_db_df.sum(axis=1)>110]
print(len(tsne_db_df_50))
tsne_db_df_50_melted = pd.melt(tsne_db_df_50,value_vars=["Unlabelled","Blast"],id_vars="Cluster Label")
ax = sns.barplot(hue="variable",y="value",x="Cluster Label",data=tsne_db_df_50_melted,log='y')
plt.savefig('e21_largeR_clstrs_only__barplot.png')
plt.close()


10
